# Agenda: Dates and times in Pandas!

1. Dates and times in programming
2. Parsing dates with Pandas
3. The `.dt` accessor
4. Loading multiple files into a single data frame
5. How can you parse odd date formats?
6. Grouping with dates
7. Time deltas
8. Time series
9. Resampling

# Dates and times in programming

The most natural (and reasonable) way to to think about dates and times is as specific, unique moments in time. I can identify a "datetime" at various points in my life, and each of these is unique:

- When I was born
- When I graduated from university
- When this meeting started
- When this meeting ends

In programming, we describe these as "datetime" or "timestamp" objects.

We also talk about spans of time. Here, there is no hour/minute/second or year/month/day. Rather, it's "10 minutes long" or "3 days long."  Some examples:

- This meeting will last for 1 hour
- My life, so far, has been about 51.5 years.
- The pa